## Special Topics in Information Systems
### Lab 2


# Face Recognition


#### by: Youssef Sherif ID: 72, and Ramez Maher ID: 27

### Problem Statement:

It is required to perform facial recognition -tell the subject's id given an image of him/her- on a database of 40 subjects. Each subject has 10 images in the database.

In [25]:
from skimage.io import imread, imshow
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os

#### Generating data matrix and labels vector

In [30]:
DATAPATH = "data"

features_vec = []
labels = []
for i in range(1, 41):
    path = os.path.join(DATAPATH, "s"+str(i))
    for img in os.listdir(path):
        image = imread(os.path.join(path, img), as_gray=True)
        features_vec.append(image.flatten())
        labels.append(i)

data_matrix = np.array(features_vec)

(400, 10304)
400


#### Spliting dataset into train and test sets

In [34]:
x1_train = []
x1_test = []
y1_train = []
y1_test = []

#Primary train and test sets split 50/50
for i in range(0, 400):
    if i%2 == 1:
        x1_train.append(data_matrix[i])
        y1_test.append(labels[i])
    else:
        x1_test.append(data_matrix[i])
        y1_test.append(labels[i])
x1_train = np.array(x1_train)
x1_test = np.array(x1_test)

#Different train and test sets splits 70/30
x2_train, x2_test, y2_train, y2_test = train_test_split(data_matrix, labels, test_size=0.3)

(200, 10304)
(280, 10304)
